# Toronto Neighbourhoods - geocodes
#### This is part of the Course <u>*Applied Data Science Capstone*</u> on Coursera, to complete the Specialization <u>*IBM Data Science Professional Certificate*</u>

This exercise is to get the geocodes from the Toronto Neighbourhoods we got in the first [notebook](https://github.com/rareal/Coursera_Capstone/blob/master/Toronto_Neighborhoods.ipynb). We're getting the latitude and longitude from the postcodes in the dataframe. 
The instructions suggest to use the `geocoder` package, but that is not working. After looking around I found https://my.locationiq.com, got a developer token, 10000 free calls a day. **LocationIQ** [api docs](https://locationiq.com/docs#forward-geocoding).


[Other googlemaps alternatives](http://geoawesomeness.com/google-maps-api-alternatives-best-cheap-affordable/).

---------------
Importing dependencies:

In [72]:
import requests
import pandas as pd
import numpy as np
import time

In [55]:
apikey = '3519d86646e89c'

LocationIQ search api call example:

In [56]:
url = "https://us1.locationiq.com/v1/search.php"
data = {
    'key': apikey,
    'q': 'Empire State Building',
    'format': 'json'
}
response = requests.get(url, params=data)

In [57]:
print(response.json()[0]['display_name'])
print('latitude: ',response.json()[0]['lat'])
print('longitude: ',response.json()[0]['lon'])

Empire State Building, 350, 5th Avenue, Korea Town, Midtown South, Manhattan, Manhattan Community Board 5, New York County, New York City, New York, 10001, USA
latitude:  40.7484284
longitude:  -73.9856546198733


got the `Toronto_Neighbourhoods.csv` from the first notebook [Toronto_Neighborhoods.ipynb](https://github.com/rareal/Coursera_Capstone/blob/master/Toronto_Neighborhoods.ipynb)

In [30]:
Toronto_neigh = pd.read_csv('Toronto_Neighbourhoods.csv')

In [75]:
nrow = len(Toronto_neigh.Postcode)
lat = pd.Series(['None']*nrow)
lon = pd.Series(['None']*nrow)

The API limit is 1 request per second, I'm doing 2s intervals between calls to be sure.

In [73]:
for i in range(nrow):
    print('iteration: ',i)
    PC = Toronto_neigh.Postcode[i]
    data = {'key': apikey,'q':'{}, Toronto, Ontario'.format(PC),'format': 'json'}
    while(lat[i] is 'None'):
    response = requests.get(url, params=data)
    lat[i] = response.json()[0]['lat']
    lon[i] = response.json()[0]['lon']
    time.sleep(2)


In [74]:
lat

array([43.754227 , 43.724686 , 43.6636637, 43.721307 , 43.653963 ,
       43.666609 , 43.653963 , 43.743931 , 43.707087 , 43.657181 ,
       43.707651 , 43.650381 , 43.653963 , 43.7328216, 43.690678 ,
       43.651386 , 43.68928  , 43.6449033, 43.766199 , 43.676218 ,
       43.6449033, 43.687783 , 43.6449033, 43.71126  , 43.657246 ,
       43.668236 , 43.76807  , 43.801989 , 43.7561987, 43.704224 ,
       43.649692 , 43.665276 , 43.744488 , 43.6449033, 43.6449033,
       43.686567 , 43.6449033, 43.648806 , 43.731742 , 43.774102 ,
       43.738576 , 43.68009  ,  0.       , 43.6449033, 43.712173 ,
       43.757762 , 43.727586 , 43.67089  , 43.64877  , 43.713895 ,
       43.762474 , 43.723138 , 43.7635305, 43.728584 , 43.657016 ,
       43.732726 , 43.697054 , 43.740315 , 43.704081 , 43.6449033,
       43.758752 , 43.729413 , 43.710815 , 43.6449033, 43.709229 ,
       43.7596   , 43.752068 , 43.713691 , 43.69855  , 43.6449033,
       43.693021 , 43.751725 , 43.779338 , 43.71275  , 43.6715

In [66]:
Toronto_neigh.Postcode[5:7]

5    M9A
6    M1B
Name: Postcode, dtype: object

In [67]:
PC='M9A'
data = {'key': apikey,'q':'{}, Toronto, Ontario'.format(PC),'format': 'json'}
response = requests.get(url, params=data)

In [71]:
response.json()[0]['lat']

'43.666609'